In [1]:
import numpy as np
import os
import pickle
from plotting_utils import heatmap,annotate_heatmap
list_of_tasks = 'autoencoder curvature denoise edge2d edge3d \
keypoint2d keypoint3d colorization \
reshade rgb2depth rgb2mist rgb2sfnorm \
room_layout segment25d segment2d vanishing_point \
segmentsemantic class_1000 class_places inpainting_whole'

task_list = 'autoencoder curvature denoise edge2d edge3d \
keypoint2d keypoint3d  \
reshade rgb2depth rgb2mist rgb2sfnorm \
room_layout segment25d segment2d vanishing_point_well_defined \
segmentsemantic_rb class_1000'

task_list_refined = 'autoencoder curvature denoise edge2d edge3d \
keypoint2d keypoint3d  \
reshade rgb2depth rgb2mist rgb2sfnorm \
room_layout segment25d segment2d vanishing_point \
semantic_seg class_1000'

task_list = task_list.split()
task_list_refined = task_list_refined.split()

with open('./../all_affinities_tk.pkl', 'rb') as f:
    data = pickle.load(f)
    
t_affinity_matrix = np.zeros((len(task_list),len(task_list)))
for i,task1 in enumerate(task_list):
    for j,task2 in enumerate(task_list):
        t_affinity_matrix[i,j] = data[task1+"__"+task2]
        
with open('./../wins_vs_pixels_16k.pkl', 'rb') as f:
    data = pickle.load(f)
#print(data)
#print(data['win_rates']['segmentsemantic_rb'])
import numpy as np
t_winrate_matrix = np.zeros((len(task_list),len(task_list)))
for i,task1 in enumerate(task_list):
    for j,task2 in enumerate(task_list):
        t_winrate_matrix[i,j] = data[task1+"__"+task2]
print(t_winrate_matrix.shape)

(17, 17)


In [2]:
results_path = './../results/DDScomparison_taskonomy_5000/'

In [3]:
from scipy.stats import spearmanr,pearsonr

kernel_affinities =  np.load(results_path+'kernels.npy',allow_pickle=True).item()
rdm_affinities = np.load(results_path+'rdms.npy',allow_pickle=True).item()

corr_value = {}
for kernel,kernel_affinity in kernel_affinities.items():
    corr_value[kernel] = {}
    for f_ablation,f_ablation_affinity in kernel_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_winrate_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] = temp
        corr_value[kernel][f_ablation] = corr_value_matrix
            

for rdm,rdm_affinity in rdm_affinities.items():
    corr_value[rdm] = {}
    for f_ablation,f_ablation_affinity in rdm_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_winrate_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] =  temp
        corr_value[rdm][f_ablation] = corr_value_matrix

In [4]:
max_corr = 0
mean_corr = {}
for dist,corr_dist in corr_value.items():
    mean_corr[dist] = {}
    for f_ablation,corr_f_ablation in corr_dist.items():
        mean_corr[dist][f_ablation] = round(np.mean(corr_f_ablation),3)
        if np.mean(corr_f_ablation) > max_corr:
            max_corr = np.mean(corr_f_ablation)
            max_combo = [dist,f_ablation]
print(max_corr,max_combo)

0.8381203313797776 ['cosine', 'znorm']


In [5]:
import pandas as pd
dfObj = pd.DataFrame(mean_corr)
print(dfObj)

                 rbf    lap  linear  pearson  euclidean  cosine
None           0.754  0.766   0.570    0.777      0.628   0.678
centering      0.754  0.766   0.570    0.694      0.628   0.682
znorm          0.818  0.827   0.820    0.832      0.821   0.838
group_norm     0.743  0.767   0.782    0.782      0.764   0.782
instance_norm  0.812  0.813   0.833    0.834      0.826   0.834
layer_norm     0.738  0.762   0.777    0.777      0.766   0.777
batch_norm     0.801  0.817   0.695    0.825      0.771   0.818


In [6]:
mean_col = 0
mean_row = 0
for j,task in enumerate(task_list):
    temp_col,_ = spearmanr(t_winrate_matrix[:,j], t_affinity_matrix[:,j])
    temp_row,_ = spearmanr(t_winrate_matrix[j,:], t_affinity_matrix[j,:])
    mean_col+=temp_col/len(task_list)
    mean_row+=temp_row/len(task_list)
print(mean_col,mean_row)

0.9194340578036585 0.6765866545500134


In [7]:
from scipy.stats import spearmanr,pearsonr

kernel_affinities =  np.load(results_path+'kernels.npy',allow_pickle=True).item()
rdm_affinities = np.load(results_path+'rdms.npy',allow_pickle=True).item()

corr_value = {}
for kernel,kernel_affinity in kernel_affinities.items():
    corr_value[kernel] = {}
    for f_ablation,f_ablation_affinity in kernel_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_affinity_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] = temp
        corr_value[kernel][f_ablation] = corr_value_matrix
            

for rdm,rdm_affinity in rdm_affinities.items():
    corr_value[rdm] = {}
    for f_ablation,f_ablation_affinity in rdm_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_affinity_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] =  temp
        corr_value[rdm][f_ablation] = corr_value_matrix

In [8]:
max_corr = 0
mean_corr = {}
for dist,corr_dist in corr_value.items():
    mean_corr[dist] = {}
    for f_ablation,corr_f_ablation in corr_dist.items():
        mean_corr[dist][f_ablation] = round(np.mean(corr_f_ablation),3)
        if np.mean(corr_f_ablation) > max_corr:
            max_corr = np.mean(corr_f_ablation)
            max_combo = [dist,f_ablation]
print(max_corr,max_combo)

0.8620242214532873 ['cosine', 'znorm']


In [9]:
import pandas as pd
dfObj = pd.DataFrame(mean_corr)
print(dfObj)

                 rbf    lap  linear  pearson  euclidean  cosine
None           0.815  0.819   0.639    0.832      0.694   0.750
centering      0.815  0.819   0.639    0.749      0.694   0.740
znorm          0.842  0.860   0.839    0.860      0.846   0.862
group_norm     0.805  0.821   0.838    0.838      0.825   0.838
instance_norm  0.843  0.839   0.851    0.853      0.850   0.853
layer_norm     0.799  0.815   0.832    0.832      0.824   0.832
batch_norm     0.851  0.854   0.734    0.862      0.811   0.852
